#### Introduction:
Angina, more commonly referred to as chest pain, is the discomfort caused by the lack of oxygen-rich blood in the heart. There are a multitude of factors that have proven to increase the risk of chest pain including age, high blood pressure, smoking, physical inactivity, and more. The purpose of our project is to predict the type of chest pain using various predictor variables. In order to do this, we have chosen the UCI Heart Disease data set, which examines the presence of heart disease in patients. 

#### Preliminary exploratory data analysis:

In [1]:
library(tidyverse)
library(tidymodels)
library(repr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [2]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
download.file(url, "data.mod")

In [4]:
# Load our dataset
chest_pain_data <- read_csv("data.mod", col_name = FALSE)
slice(chest_pain_data,1:6)

Parsed with column specification:
cols(
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double(),
  X6 = col_double(),
  X7 = col_double(),
  X8 = col_double(),
  X9 = col_double(),
  X10 = col_double(),
  X11 = col_double(),
  X12 = col_character(),
  X13 = col_character(),
  X14 = col_double()
)



X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0


In [5]:
# add colomn names
names(chest_pain_data) <- c("age", "sex",
                         "chest_pain_type",
                         "trest_bps",
                          "cholesterol",
                          "fasting_blood_sugar",
                          "resting_ecg",
                          "max_heart_rate",
                          "exercise_induced_angina",
                          "oldpeak",
                          "slope",
                          "no_vessels_colored",
                          "thal",
                          "healthy")

In [6]:
# convert all categorical variables into a factor using the as_factor() function.
chest_pain_data <- mutate(chest_pain_data, sex = as_factor(sex),
                          chest_pain_type = as_factor(chest_pain_type),
                          fasting_blood_sugar = as_factor(fasting_blood_sugar),
                          resting_ecg = as_factor(resting_ecg),
                          exercise_induced_angina = as_factor(exercise_induced_angina),
                          slope = as_factor(slope),
                          no_vessels_colored = as_factor(no_vessels_colored),
                          thal = as_factor(thal),
                          healthy = as_factor(healthy)
                          )
slice(chest_pain_data,1:6)               

age,sex,chest_pain_type,trest_bps,cholesterol,fasting_blood_sugar,resting_ecg,max_heart_rate,exercise_induced_angina,oldpeak,slope,no_vessels_colored,thal,healthy
<dbl>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0


In [ ]:
# split our dataset into a training dataset and a testing dataset
# Randomly take 75% of the data in the training set. 
# This will be proportional to the different number of fruit names in the dataset.

chest_pain_data_split <- initial_split(fruit_data, prop = 0.75, strata =... )  
chest_pain_data_train <- training(chest_pain_data_split)   
chest_pain_data_test <- testing(chest_pain_data_split)

#### Methods:
We plan to use the method of classification to predict the type of chest pain (typical angina, atypical angina, non-anginal pain, or asymptomatic). First we will split our data into a training and testing dataset, and then use cross-validation to evaluate different choices of K in the K-nearest neighbors algorithm. For our data analysis we will be predicting the variable of chest pain type, using the columns: age, chest pain type, serum cholesterol, maximum heart rate, and resting blood pressure.

First, we will visualize the results using a scatter plot and plot 2 variables, age and cholesterol, with the 4 chest pain types color coordinated. By choosing these two as our initial variables, we can easily draw any trends or patterns from the plot and determine if there are any similarities between each point and its neighboring points. As we add more predictor variables into our classification analysis, we can determine which value of k is most accurate by using cross-validation on our training set and plotting k against accuracy in a cross-validation plot. Lastly, we can use a confusion matrix to report the accuracy of the predictions of our final model on the test dataset.


#### Expected outcomes and significance: 
We expect to find which variables are the most important for chest pain type. Looking at each variable, we expect that the higher the age, blood pressure, cholesterol level, and maximum heart rate should result in a higher level of chest pain. We are predicting that the variables of age and cholesterol levels to be the most accurate predictors in deciding what chest pain level we have.

These findings could impact the health industry concerns with chest pains in important ways. If we know which variables are likely to cause severe chest pain, then patients with this kind of chest pain know to be more cautious of specific things. For example, if we were to find that having a high resting blood pressure likely corresponds to typical angina, patients with this type of chest pain should more closely watch their blood pressure levels.

Some future questions this could lead to include:
* What other factors may affect chest pain?
* Are we able to do this analysis with other pains, such as shoulder pain?
* Could these same variables also link to other health issues in the body?
* Are there any omitted variables in our classification analysis that may be influencing our results?
